In [ ]:
%pip install --upgrade pip setuptools wheel disutils

In [ ]:
%pip install numpy pandas matplotlib seaborn scikit-learn tensorflow obspy

In [ ]:
%pip install eqtransformer

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from obspy import read

# Define el directorio de datos
path_to_dataset = r'C:\Users\Kono\Desktop\space_apps_2024_seismic_detection\data'
sys.path.append(path_to_dataset)
data_directory = path_to_dataset + r'\lunar\training\data\S12_GradeA'
data_files = os.listdir(data_directory)
data_files = [data_directory + '\\' + file for file in data_files if file.endswith('.mseed')]
abstract_dfs = pd.read_csv(r'C:\Users\Kono\Desktop\space_apps_2024_seismic_detection\data\lunar\training\catalogs\apollo12_catalog_GradeA_final.csv')

# Inicializa un DataFrame vacío para almacenar los resultados
df = pd.DataFrame(columns=['file_name', 'start', 'id', 'cant_measurements', 'st'])

for index, file in enumerate(data_files):
    print(f'Processing file {index+1} of {len(data_files)}')
    
    # Lee el archivo .mseed
    temp_df = read(file)
    
    # Extrae el ID del evento del nombre del archivo
    evid_id = file.split('\\')[-1].split('_')[-1].split('evid')[1].split('.')[0]
    
    # Extrae el nombre del archivo sin la extensión .mseed
    file_name = file.split('\\')[-1].rstrip('.mseed')
    
    # Verifica si el nombre del archivo existe en el DataFrame abstract_dfs
    if len(abstract_dfs[abstract_dfs['filename'] == file_name]['time_rel(sec)']) == 0:
        continue  # Salta este archivo si no existe en abstract_dfs
    
    # Obtiene el tiempo de inicio del DataFrame abstract_dfs
    start = abstract_dfs[abstract_dfs['filename'] == file_name]['time_rel(sec)'].iloc[0]
    
    # Extrae la traza y los datos
    tr = temp_df.traces[0].copy()
    tr_data = tr.data  # Velocidades
    tr_times = tr.times()  # Tiempos relativos

    # Crea un diccionario temporal con la información requerida
    temp_dict = {
        'file_name': file_name, 
        'start': start,  
        'id': evid_id, 
        'cant_measurements': temp_df[0].stats.npts,
        'st': temp_df
    }

    df.to_csv("seismic_data_prepared.csv", index=False)
    # Agrega el diccionario temporal al DataFrame principal
    df = pd.concat([df, pd.DataFrame([temp_dict])], ignore_index=True)


In [ ]:
from eqtransformer import EQTransformer
from eqtransformer.core.data_loader import DataLoader

# Cargar los datos procesados
df = pd.read_csv("seismic_data_prepared.csv")

# Cargar los archivos .mseed para el DataLoader de EQTransformer
data_loader = DataLoader(df['file_name'].tolist(), df['start'].tolist())

# Crear una instancia del modelo EQTransformer para regresión
model = EQTransformer(
    model_type='regression',  # Cambiado a 'regression'
    input_shape=(1, 3000),  # Ajusta según la longitud de tus trazas
    learning_rate=0.001,
    batch_size=32,
    num_epochs=50
)

# Compilar el modelo
model.compile()


In [2]:
from scipy import signal
from matplotlib import cm

def plot_seismic_data(st, start, start_prediction):
    # Filtrar la señal
    minfreq = 0.5
    maxfreq = 1.0
    st_filt = st.copy()
    st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    tr_times_filt = tr_filt.times()
    tr_data_filt = tr_filt.data
    
    f, t, sxx = signal.spectrogram(tr_data_filt, tr_filt.stats.sampling_rate)

    # Graficar la serie temporal y el espectrograma
    fig = plt.figure(figsize=(10, 10))
    
    ax1 = plt.subplot(2, 1, 1)
    ax1.plot(tr_times_filt, tr_data_filt)
    
    # Marcar el inicio de la predicción
    ax1.axvline(x=start_prediction, color='green', label='Predicción de inicio')
    ax1.axvline(x=start, color='red', label='Inicio real')
    
    ax1.legend(loc='upper left')
    ax1.set_xlim([min(tr_times_filt), max(tr_times_filt)])
    ax1.set_ylabel('Velocidad (m/s)')
    ax1.set_xlabel('Tiempo (s)')
    
    ax2 = plt.subplot(2, 1, 2)
    vals = ax2.pcolormesh(t, f, sxx, cmap=cm.jet, vmax=5e-17)
    ax2.set_xlim([min(tr_times_filt), max(tr_times_filt)])
    ax2.set_xlabel('Tiempo (s)', fontweight='bold')
    ax2.set_ylabel('Frecuencia (Hz)', fontweight='bold')
    
    # Marcar el inicio de la predicción en el espectrograma
    ax2.axvline(x=start_prediction, color='green')
    ax2.axvline(x=start, color='red')

    # Añadir barra de color
    cbar = plt.colorbar(vals, orientation='horizontal')
    cbar.set_label('Power ((m/s)^2/sqrt(Hz))', fontweight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
i = 6
plot_seismic_data(df.iloc[i]['st'], df.iloc[i]['start'], df.iloc[i]['start'] - 10000)

In [4]:
import numpy as np
from scipy import signal

# Prepare los datos para el modelo
spectrograms = []
starts = []

for index, row in df.iterrows():
    st = row['st']
    
    # Filtrar la señal
    minfreq = 0.5
    maxfreq = 1.0
    st_filt = st.copy()
    st_filt.filter('bandpass', freqmin=minfreq, freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    
    # Calcular el espectrograma
    f, t, sxx = signal.spectrogram(tr_filt.data, tr_filt.stats.sampling_rate)
    sxx = np.transpose(sxx)
    # Añadir espectrograma y start a las listas

    spectrograms.append(sxx)
    starts.append(row['start'])

# Encontrar el tamaño máximo de los espectrogramas
max_height = max(s.shape[0] for s in spectrograms)
max_width = max(s.shape[1] for s in spectrograms)

# Rellenar los espectrogramas con ceros para uniformizar
padded_spectrograms = []
for sxx in spectrograms:
    # Rellenar la matriz con ceros hasta el tamaño máximo
    pad_width = ((0, max_height - sxx.shape[0]), (0, max_width - sxx.shape[1]))
    padded_spectrograms.append(np.pad(sxx, pad_width, mode='constant'))

# Convertir listas a arrays numpy
X = np.array(padded_spectrograms)
y = np.array(starts)


In [ ]:
# Verificar la longitud de ambas listas
print(X.shape, y.shape)


In [ ]:
# Redefinir la forma de los datos para la entrada en el modelo
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Añadir dimensión para canales

# Crear el modelo CNN
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)  # Salida única para la predicción del valor 'start'
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model.fit(X, y, epochs=3, batch_size=32)  # Ajustar según sea necesario


In [ ]:
# Evaluar el modelo
loss = model.evaluate(X, y)
print(f'Loss: {loss}')


In [ ]:
predictions = model.predict(X)

# Comparar las predicciones usando la función plot_seismic_data
for i in range(len(predictions)):
    plot_seismic_data(df.iloc[i]['st'], df.iloc[i]['start'], predictions[i][0])
    break

